In [10]:
from __future__ import print_function
import os
import sys
import pickle
from collections import Counter, OrderedDict
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [2]:
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu'

In [3]:
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
from keras.models import load_model

Using Theano backend.


In [4]:
# Read image file_name and their captions
file_name_caption = pickle.load(open( "file_name_caption.bat", "rb" ))
# Read image file_name and their VGG 16 weights
file_name_images = pickle.load(open( "file_name_images.bat", "rb" ))

In [13]:
keras_model = 'keras_image_caption_model.dat'
train_dir = '/home/qhduan/Downloads/imagedata/train2014'
val_dir = '/home/qhduan/Downloads/imagedata/val2014'

In [14]:
model = load_model(keras_model)

In [5]:
argument = pickle.load(open('argument.dat', 'rb'))

In [6]:
word_index = argument['word_index']
index_word = argument['index_word']
max_len = argument['max_len']
START = argument['START']
END = argument['END']
UNK = argument['UNK']
PAD = argument['PAD']

In [7]:
def sent_to_index(input_sent, word_index, max_len):
    padding_size = max_len - len(input_sent)

    input_sent = input_sent + padding_size * [PAD]
    input_sent_index = []
    for w in input_sent:
        if w in word_index:
            input_sent_index.append(word_index[w])
        else:
            input_sent_index.append(word_index[UNK])
    return input_sent_index

In [8]:
def predict_weights(weights, word_index, index_word, max_len):
    sent = [START]
    while True:
        sent_index = sent_to_index(sent, word_index, max_len)
        index = model.predict([np.asarray([weights]), np.asarray([sent_index])]).argmax()
        if index in index_word:
            word = index_word[index]
        else:
            word = UNK
        sent.append(word)
        if word == END or len(sent) > max_len:
            break
    return sent

In [9]:
file_name_images = sorted(list(file_name_images.items()), key=lambda x: x[0])

In [18]:
train_bleu = []
for file_name, weights in tqdm(file_name_images, file=sys.stdout):
    if 'train' not in file_name:
        continue
    ret = predict_weights(weights, word_index, index_word, max_len)
    while ret[0] == START:
        ret = ret[1:]
    while ret[-1] == END:
        ret = ret[:-1]
    true = word_tokenize(file_name_caption[file_name])
    predict = ret
    b = sentence_bleu([true], predict, weights=(1.0,))
    train_bleu.append(b)
    if len(train_bleu) >= 1000:
        break


  0%|          | 3/123050 [00:02<25:56:21,  1.32it/s]


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/qhduan/.local/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



  1%|          | 999/123050 [12:03<24:27:09,  1.39it/s]


In [26]:
test_bleu = []
for file_name, weights in tqdm(file_name_images, file=sys.stdout):
    if 'val' not in file_name:
        continue
    ret = predict_weights(weights, word_index, index_word, max_len)
    while ret[0] == START:
        ret = ret[1:]
    while ret[-1] == END:
        ret = ret[:-1]
    true = word_tokenize(file_name_caption[file_name])
    predict = ret
    b = sentence_bleu([true], predict, weights=(1.0,))
    test_bleu.append(b)
    if len(test_bleu) >= 1000:
        break

 68%|██████▊   | 83611/123050 [12:37<7:50:26,  1.40it/s]  

In [20]:
np.mean(train_bleu)

0.30985391884751368

In [27]:
np.mean(test_bleu)

0.306036089425132